# __키워드로 이미지 검색하기__ 

## __0. 데이터 세트 준비__

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [2]:
%%thanosql
COPY diet FROM "tutorial_data/diet_data/diet.csv"

Success


## __1. 데이터 세트 확인__

키워드-이미지 검색 모델을 만들기 위해 ThanoSQL DB에 저장되어 있는 diet 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [3]:
%%thanosql
SELECT * 
FROM diet

,image,label
0,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
1,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
2,tutorial_data/diet_data/diet/백향과/1_A220148XX_0...,백향과
3,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
4,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
...,...,...
1185,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이
1186,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이
1187,tutorial_data/diet_data/diet/사과파이/1_A020511XX_...,사과파이
1188,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이


## __2. 키워드 검색 모델 생성__ 

이미지 검색을 위해서는 기존 데이터 테이블을 학습하여 추후 검색의 기준을 만들어줘야 합니다. 이를 위해서 이전 단계에서 확인한 데이터 세트를 사용하여 이미지 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 diet_image_classification이라는 이름의 모델을 만듭니다. 

In [4]:
%%thanosql
BUILD MODEL diet_image_classification
USING ConvNeXt_Tiny
OPTIONS (
    image_col='image', 
    label_col='label', 
    epochs=1
    )
AS 
SELECT *
FROM diet

Success


## __3. 생성된 모델을 사용하여 키워드-이미지 검색 모델 확인__

이전 단계에서 만든 이미지 예측 모델 diet_image_classification을 사용해서 특정 이미지의 목표값을 예측해 봅니다. 아래 쿼리를 수행하고 나면, 예측 결과는 predicted 컬럼에 저장되어 반환됩니다.

In [5]:
%%thanosql
PREDICT USING diet_image_classification
AS 
SELECT *
FROM diet

,image,label,predicted
0,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
1,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,백향과
2,tutorial_data/diet_data/diet/백향과/1_A220148XX_0...,백향과,복숭아
3,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
4,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
...,...,...,...
1185,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
1186,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,빵
1187,tutorial_data/diet_data/diet/사과파이/1_A020511XX_...,사과파이,보쌈
1188,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,복숭아


## __4. 생성된 모델을 이용한 검색__ 

이제 "PREDICT USING", "SELECT", "WHERE" 쿼리 구문을 사용하여 특정 조건의 데이터만을 검색합니다. label이 '사과파이'이고, 예측 결과 또한 '사과파이'인 데이터만을 검색하고 다음처럼 쿼리 구문을 작성할 수 있습니다.

In [6]:
%%thanosql
SELECT * 
FROM (
    PREDICT USING diet_image_classification
    AS 
    SELECT *
    FROM diet
    )
WHERE label = predicted
AND label LIKE '사과파이'
LIMIT 10

,image,label,predicted
0,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
1,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
2,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
3,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
4,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
5,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
6,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
7,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
8,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
9,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
